## Problem Statement

### Business Context

Business communities in the United States are facing high demand for human resources, but one of the constant challenges is identifying and attracting the right talent, which is perhaps the most important element in remaining competitive. Companies in the United States look for hard-working, talented, and qualified individuals both locally as well as abroad.

The Immigration and Nationality Act (INA) of the US permits foreign workers to come to the United States to work on either a temporary or permanent basis. The act also protects US workers against adverse impacts on their wages or working conditions by ensuring US employers' compliance with statutory requirements when they hire foreign workers to fill workforce shortages. The immigration programs are administered by the Office of Foreign Labor Certification (OFLC).

OFLC processes job certification applications for employers seeking to bring foreign workers into the United States and grants certifications in those cases where employers can demonstrate that there are not sufficient US workers available to perform the work at wages that meet or exceed the wage paid for the occupation in the area of intended employment.

### Objective

In FY 2016, the OFLC processed 775,979 employer applications for 1,699,957 positions for temporary and permanent labor certifications. This was a nine percent increase in the overall number of processed applications from the previous year. The process of reviewing every case is becoming a tedious task as the number of applicants is increasing every year.

The increasing number of applicants every year calls for a Machine Learning based solution that can help in shortlisting the candidates having higher chances of VISA approval. OFLC has hired the firm EasyVisa for data-driven solutions. You as a data  scientist at EasyVisa have to analyze the data provided and, with the help of a classification model:

* Facilitate the process of visa approvals.
* Recommend a suitable profile for the applicants for whom the visa should be certified or denied based on the drivers that significantly influence the case status.

### Data Description

The data contains the different attributes of employee and the employer. The detailed data dictionary is given below.

* case_id: ID of each visa application
* continent: Information of continent the employee
* education_of_employee: Information of education of the employee
* has_job_experience: Does the employee has any job experience? Y= Yes; N = No
* requires_job_training: Does the employee require any job training? Y = Yes; N = No
* no_of_employees: Number of employees in the employer's company
* yr_of_estab: Year in which the employer's company was established
* region_of_employment: Information of foreign worker's intended region of employment in the US.
* prevailing_wage:  Average wage paid to similarly employed workers in a specific occupation in the area of intended employment. The purpose of the prevailing wage is to ensure that the foreign worker is not underpaid compared to other workers offering the same or similar service in the same area of employment.
* unit_of_wage: Unit of prevailing wage. Values include Hourly, Weekly, Monthly, and Yearly.
* full_time_position: Is the position of work full-time? Y = Full Time Position; N = Part Time Position
* case_status:  Flag indicating if the Visa was certified or denied

## Installing and Importing the necessary libraries

In [4]:
# Installing the libraries with the specified version.
!pip install numpy==1.25.2 pandas==1.5.3 scikit-learn==1.5.2 matplotlib==3.7.1 seaborn==0.13.1 xgboost==2.0.3 -q --user

**Note**: *After running the above cell, kindly restart the notebook kernel and run all cells sequentially from the below.*

In [5]:
# Import all the necessary libraries

import warnings

warnings.filterwarnings("ignore")

# Libraries to help with reading and manipulating data
import numpy as np
import pandas as pd

# Library to split data
from sklearn.model_selection import train_test_split

# To oversample and undersample data
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score


# libaries to help with data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Removes the limit for the number of displayed columns
pd.set_option("display.max_columns", None)
# Sets the limit for the number of displayed rows
pd.set_option("display.max_rows", 100)


# Libraries different ensemble classifiers
from sklearn.ensemble import (
    BaggingClassifier,
    RandomForestClassifier,
    AdaBoostClassifier,
    GradientBoostingClassifier
)

from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

# Libraries to get different metric scores
from sklearn import metrics
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)

# To tune different models
from sklearn.model_selection import RandomizedSearchCV

## Import Dataset

In [7]:
visa = pd.read_csv('/content/EasyVisa.csv') ##  Fill the blank to read the data

FileNotFoundError: [Errno 2] No such file or directory: '/content/EasyVisa.csv'

In [ ]:
# copying data to another variable to avoid any changes to original data
data = visa.copy()

## Overview of the Dataset

#### View the first and last 5 rows of the dataset

In [ ]:
# Code to view the top 5 rows of the data
data.head()

In [ ]:
# Code to view the top 5 rows of the data
data.tail()

#### Understand the shape of the dataset

In [ ]:
# Shape depicts the number of rows and columns of the the file
# Understanding the number of rows and columns in the given dataset
print("There are", data.shape[0], 'rows and', data.shape[1], "columns.")

#### Check the data types of the columns for the dataset

In [ ]:
# Understanding the data types of the columns
data.info()

There are overall 12 columns, with 9 of them as String or object data type, 2 are of integer and 1 field is of decimal float64 data type.

In [ ]:
# Check if there are no duplicate values, across the row values in the dataset
data.duplicated().sum()

The above code implies the presence of no duplicate rows in the dataset

In [ ]:
# Check if there are any missing values in the dataset provided
data.isnull().sum()

There are no missing values in the dataset

In [ ]:
# Alternatively, this code below can be checked with FALSE indicating there are no missing values
data.isnull().values.any()

## Exploratory Data Analysis (EDA)

#### Let's check the statistical summary of the data

In [ ]:
# Describe function on the data file would provide all the statistical info on all the data fields
data.describe(include = 'all').T

The dataset has 25480 rows and 12 columns.

No_of_employees, yr_of_estab, and prevailing_wage are numeric features while rest are objects.

The no of employees has a mean of 5,667 and median of 2,109. This indicates that the distribution is skewed, to be very spcific as right-skewed because the mean is greater than median value.

There are negative values in the no of employees which could be an error.

Yr of estab ranges from 1800 to 2016, with it's mean value as 1979.40 > median or 50% value at 1997, indicates the data is right skewed and prevailing wage ranges from 2 to 319,210, whose distribution is also right skewed because of its mean = 74455.81 > median = 70308.21.

The most popular continent is Asia

The most popular educational level is Bachelor

#### Fixing the negative values in number of employees columns

In [ ]:
# Count the number of employee records which are incorreclty depicted as a negative number
data.loc[data['no_of_employees']<0].shape

There are overall 33 such records with negative count of number of employees

Which in in real-time not possible and is hence, treated to be a data entry error and hence, modulus of such records is taken in order to treat these negative values as absolute numbers instead.

In [ ]:
# Converting the negative values to absolute positive values
data['no_of_employees'] = abs(data['no_of_employees'])

In [ ]:
# Checking Describe function again on the data file, to find out the presence of any negative values in the min value of the row/field: no_of_employees
data.describe(include = 'all').T

* No_of_employees field's negative values have been converted to positive values, as indicated by the new min value as 11 instead of -26 shown previously

#### Let's check the count of each unique category in each of the categorical variables

In [ ]:
# categorial columns
obj_col = list(data.select_dtypes("object").columns)

# Printing number of count of each unique value in each column
for column in obj_col:
    print(data[column].value_counts(True))
    print("-" * 50)

Case id can be removed as it is unique value and does not contribute to the analysis.

66.17% of the applications are from Asians.

40.16% of the applicants have a bachelor's degree.

58.09% of the applicants have job experience.

88.40% of the applicants do not require job training.

28.23% of the applicants have their worksite in the Northeast region of the US.

90.11% of the applicants have a yearly unit of wage. While it has 4 unique values.

89.37% of the visa applications are having a full-time job positions

66.78% of the case status are certified, while the remaining 33.21% are denied, hence making the target variable imbalanced.

* Unit_of_wage field is in different units, hence needs to converted into one scale for easy measurements.

In [ ]:
# checking the number of unique values using the unique function
data["case_id"].nunique()

* Case_id field has unique values of 25480 count, which is exactly the same as the number of records in the visa file given and hence the field with unique records does not have any significance in the analysis and hence, can be dropped as below from the data file

In [ ]:
# code to drop 'case_id' column from the data
data.drop(['case_id'], axis=1, inplace=True)

In [ ]:
# Check that the column was dropped
data.head()

* 'case_id' column was dropped successfully

### Univariate Analysis

In [ ]:
# Function to plot boxplot and histogram
def histogram_boxplot(data, feature, figsize=(15, 10), kde=False, bins=None):
    """
    Boxplot and histogram combined

    data: dataframe
    feature: dataframe column
    figsize: size of figure (default (15,10))
    kde: whether to show the density curve (default False)
    bins: number of bins for histogram (default None)
    """
    f2, (ax_box2, ax_hist2) = plt.subplots(
        nrows=2,  # Number of rows of the subplot grid= 2
        sharex=True,  # x-axis will be shared among all subplots
        gridspec_kw={"height_ratios": (0.25, 0.75)},
        figsize=figsize,
    )  # creating the 2 subplots
    sns.boxplot(
        data=data, x=feature, ax=ax_box2, showmeans=True, color="violet"
    )  # boxplot will be created and a triangle will indicate the mean value of the column
    sns.histplot(
        data=data, x=feature, kde=kde, ax=ax_hist2, bins=bins
    ) if bins else sns.histplot(
        data=data, x=feature, kde=kde, ax=ax_hist2
    )  # For histogram
    ax_hist2.axvline(
        data[feature].mean(), color="green", linestyle="--"
    )  # Add mean to the histogram
    ax_hist2.axvline(
        data[feature].median(), color="black", linestyle="-"
    )  # Add median to the histogram

In [ ]:
# function to create labeled barplots
def labeled_barplot(data, feature, perc=False, n=None):
    """
    Barplot with percentage at the top

    data: dataframe
    feature: dataframe column
    perc: whether to display percentages instead of count (default is False)
    n: displays the top n category levels (default is None, i.e., display all levels)
    """

    total = len(data[feature])  # length of the column
    count = data[feature].nunique()
    if n is None:
        plt.figure(figsize=(count + 2, 6))
    else:
        plt.figure(figsize=(n + 2, 6))

    plt.xticks(rotation=90, fontsize=15)
    ax = sns.countplot(
        data=data,
        x=feature,
        palette="Paired",
        order=data[feature].value_counts().index[:n],
    )

    for p in ax.patches:
        if perc == True:
            label = "{:.1f}%".format(
                100 * p.get_height() / total
            )  # percentage of each class of the category
        else:
            label = p.get_height()  # count of each level of the category

        x = p.get_x() + p.get_width() / 2  # width of the plot
        y = p.get_height()  # height of the plot

        ax.annotate(
            label,
            (x, y),
            ha="center",
            va="center",
            size=12,
            xytext=(0, 5),
            textcoords="offset points",
        )  # annotate the percentage

    plt.show()  # show the plot

#### Observations on education of employee

In [ ]:
# Bar plat on 'Education_of_employee' field
labeled_barplot(data, "education_of_employee", perc=True)

40.2% of the applicants have a bachelor's degree, followed by 37.8% having a master's degree.
8.6% of the applicants have a doctorate degree, while 13.4% of visa applicants have completed only High School.

#### Observations on region of employment

In [ ]:
# Bar plat on 'Education_of_employee' field
labeled_barplot(data, "region_of_employment", perc=True)

Northeast, South, and West are around 25% of distribution.

The Island regions have only 1.5% of the applicants, while the Midwest region has 16.9% of applicants.

#### Observations on job experience

In [ ]:
# Bar plot on 'Has Job Experience' field
labeled_barplot(data, "has_job_experience", perc=True)

58.1% of the applicants have job experience, while 41.9% of the applicants do not have job experience.

#### Observations on case status

In [ ]:
# Bar plot on 'Case status' field
labeled_barplot(data, "case_status", perc=True)

66.8% of the visas were certified, while 33.2% were denied.

In [ ]:
# Histogram plot on prevailing wage
histogram_boxplot(data, 'prevailing_wage')

There is a big variation on prevailing wage, coulod be attributed to the unit of wage field, as there were 4 unique values.
The mean and median are close together around 75000, however many outliers are observed on the upper side / right side on this right skewed distribution.

In [ ]:
# Bar plot on the 'Unit_of_wage' field
labeled_barplot(data, 'unit_of_wage',perc=True)

Most of the wages are represented yearly (90.1%). 8.5% of the wages are hourly.
Very few unit of wage are weekly or monthly at 1.1 % or 0.3% respectively - this throws an interesting argument on the conversion of all the unit of wages into Yearly.

But the unit of wage, could be even attributed to the nature of the job undertaken and hence converting to one scale would need a detailed discussion with the Business and then performing it, rathern than doing it NOW !

### Bivariate Analysis

**Creating functions that will help us with further analysis.**

In [ ]:
### function to plot distributions wrt target


def distribution_plot_wrt_target(data, predictor, target):

    fig, axs = plt.subplots(2, 2, figsize=(12, 10))

    target_uniq = data[target].unique()

    axs[0, 0].set_title("Distribution of target for target=" + str(target_uniq[0]))
    sns.histplot(
        data=data[data[target] == target_uniq[0]],
        x=predictor,
        kde=True,
        ax=axs[0, 0],
        color="teal",
        stat="density",
    )

    axs[0, 1].set_title("Distribution of target for target=" + str(target_uniq[1]))
    sns.histplot(
        data=data[data[target] == target_uniq[1]],
        x=predictor,
        kde=True,
        ax=axs[0, 1],
        color="orange",
        stat="density",
    )

    axs[1, 0].set_title("Boxplot w.r.t target")
    sns.boxplot(data=data, x=target, y=predictor, ax=axs[1, 0], palette="gist_rainbow")

    axs[1, 1].set_title("Boxplot (without outliers) w.r.t target")
    sns.boxplot(
        data=data,
        x=target,
        y=predictor,
        ax=axs[1, 1],
        showfliers=False,
        palette="gist_rainbow",
    )

    plt.tight_layout()
    plt.show()

In [ ]:
def stacked_barplot(data, predictor, target):
    """
    Print the category counts and plot a stacked bar chart

    data: dataframe
    predictor: independent variable
    target: target variable
    """
    count = data[predictor].nunique()
    sorter = data[target].value_counts().index[-1]
    tab1 = pd.crosstab(data[predictor], data[target], margins=True).sort_values(
        by=sorter, ascending=False
    )
    print(tab1)
    print("-" * 120)
    tab = pd.crosstab(data[predictor], data[target], normalize="index").sort_values(
        by=sorter, ascending=False
    )
    tab.plot(kind="bar", stacked=True, figsize=(count + 5, 5))
    plt.legend(
        loc="lower left", frameon=False,
    )
    plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
    plt.show()

In [ ]:
# seperate the numerical values, for Heat Map analysis
cols_list = data.select_dtypes(include=np.number).columns.tolist()

# create the correlation matrix
plt.figure(figsize=(10, 5))
sns.heatmap(
    data[cols_list].corr(), annot=True, vmin=-1, vmax=1, fmt=".2f", cmap="Spectral"
)
plt.show()

* We could not find an correlation between the numerical field values as no_of_employees, ry_of_estab and prevailing_wage

#### Those with higher education may want to travel abroad for a well-paid job. Let's find out if education has any impact on visa certification

In [ ]:
# Relation between education and case status
stacked_barplot(data, 'education_of_employee', 'case_status')

* There is a strong relationship between visa approvals and education level.

The higher the education, the approval rate is higher.

Workers with a High School degree only have around 30% chance of approval.

Workers with a Bachelor's degree or higher (Master's or Doctorate) have a minimum of 65% approval rate.

#### Lets' similarly check for the continents and find out how the visa status vary across different continents.

In [ ]:
# Relation between continent and case status
stacked_barplot(data, 'continent', 'case_status')

* South America has the highest denial rate around 42%, while Europe has the highest approval rate with ~80% of the visas being certified. While all the other Continents are between the above two continents in terms of applicants being certified.

#### Experienced professionals might look abroad for opportunities to improve their lifestyles and career development. Let's see if having work experience has any influence over visa certification

In [ ]:
# Relation between job experience and case status
stacked_barplot(data, 'has_job_experience', 'case_status')

* There is about 20% increase in the chance for the applicant to be Certified, if the applicant has prior Job experience

#### Checking if the prevailing wage is similar across all the regions of the US

In [ ]:
# Box plot comparing the two fields as region of employment and prevailing wage
plt.figure(figsize=(10, 5))
sns.boxplot(data=data, x="region_of_employment", y="prevailing_wage")
plt.show()

* Prevailing wages is higher in Midwest and Island, as opposed to other regions.

#### The US government has established a prevailing wage to protect local talent and foreign workers. Let's analyze the data and see if the visa status changes with the prevailing wage

In [ ]:
# Relation between prevailing wage and case status
distribution_plot_wrt_target(data, 'prevailing_wage', 'case_status')

* There is not much of difference between certified and denied applicants w.r.t prevailing wages.
* While in the case of with or without outliers, the median of prevailing wages of denied cases is slightly lower than those of the Certified applicants.

#### The prevailing wage has different units (Hourly, Weekly, etc). Let's find out if it has any impact on visa applications getting certified.

In [ ]:
# Relation between unit of wage vs case status
stacked_barplot(data, "unit_of_wage", "case_status")

* Hourly waged applicants are least likely to be certified, while those with yearly are highly likely to be certified.
* Because of this relationship with the status of the visa to the unit of wages, it would be better to keep unit of wage field AS IS !!

## Data Pre-processing

### Outlier Check

In [ ]:
# outlier detection using boxplot
numeric_columns = data.select_dtypes(include=np.number).columns.tolist()


plt.figure(figsize=(15, 12))

for i, variable in enumerate(numeric_columns):
    plt.subplot(4, 4, i + 1)
    plt.boxplot(data[variable], whis=1.5)
    plt.tight_layout()
    plt.title(variable)

plt.show()

* There are few outliers in these numerical values
* However, they could be original values as intended and call for a discussion with the Business, instead of being treated as such !

### Feature Engineering

* We cannot make much of anlaysis with the Year values and hence, it would be better to arrive at a field no_of_years_since_estab instead of yr_of_estab field

In [ ]:
# Checking the highest year of establishment
data.yr_of_estab.max()

In [ ]:
# Modifying the year of establishment to have the new column incorporated
data['no_of_years_since_estab'] = 2017 - data['yr_of_estab']
data.head()

In [ ]:
# Dropping the column year of establishment
data.drop(['yr_of_estab'], axis=1, inplace=True)
data.head()

* Just like case_id column (for it's unique values) - the column (yr_of_estab) has been dropped from the data and replaced with the field no_of_years_since_estab for further analysis and modeling activities and the data has been checked as well above !

### Data Preparation for modeling

* We want to predict the Visa Applicant who will be Certified - is the task at hand !

* Before we proceed to build a model, we'll have to encode categorical features, as the Machine Learning models can neither handle String / object variables nor categorical values, as they comprehend only numerical values

* We'll split the data into train, val and test to be able to evaluate the model that we build on the train data.

In [ ]:
# encode case status, where certified is 1, denied is 0
data["case_status"] = data["case_status"].apply(lambda x: 1 if x == "Certified" else 0)

In [ ]:
# separating the independent and dependent variables as X and Y
X = data.drop(["case_status"], axis=1)
Y = data["case_status"]

# create dummy varialbes for categories
X = pd.get_dummies(X, drop_first=True)

In [ ]:
# Splitting data into training, validation and test set:
# Split the dataset into train and valid with a ratio of 7:3
X_train, X_val, y_train, y_val = train_test_split(
    X, Y, test_size=0.3, random_state=1, stratify=Y
)

# # Complete the code to split the dataset into valid and test with a ratio of 9:1
X_val,X_test,y_val,y_test = train_test_split(
    X_val,y_val,test_size=0.1,random_state=1,stratify=y_val
)


In [ ]:
# Checking class balance for whole data, train set, validation set, and test set
print("Shape of Training set : ", X_train.shape)
print("Shape of Validation set : ", X_val.shape)
print("Shape of test set : ", X_test.shape)
print("Percentage of classes in training set:")
print(y_train.value_counts(normalize=True))
print("Percentage of classes in validation set:")
print(y_val.value_counts(normalize=True))
print("Percentage of classes in test set:")
print(y_test.value_counts(normalize=True))

* Use of Startify has helped ensure the ratio of case status in all the three splits of Training, Validation and the Test data is even at 66.79% for the Certified Cases, otherwise when denied is 0

## Model Building

## Model evaluation criterion


**What does OFLC (Office of Foreign Labor Certification) want?**
* OFLC wants to certify the right applicants only:
   * Whenever OFLC deems an applicant to be certified, if the Machine Learning Model we are building predicts the applicant to be denied - implying FN (False Negative) cases have to be lowered (i.e., Recall metric to be considered) - which otherwise will lead to opportunity lost or Talent lost and Visa not provided to right talent.
   * Similarly when OFLC deems an applicant to be denied, if the Machine Learning Model we are building predicts the applicant to be certified - implying FP (False Positive) cases have to be lowered (i.e., Precision metric is to be considered) - which otherwise will be providing visa to the wrong candidate and there could be a scenario of not so talented workforce entering the Country.

**Since OFLC wants to award Certified Visa only to the right and skillfull Candidates, we have to balance and minimize the scenarios of FN and FP cases, thus implying F1 score is the right metric to be used for our model evaluation !.**

* Hence, high F1 score metric of the model will yield us the right Candidates to be certified, with both both FN and FP being lowered.


In [ ]:
# defining a function to compute different metrics to check performance of a classification model built using sklearn


def model_performance_classification_sklearn(model, predictors, target):
    """
    Function to compute different metrics to check classification model performance

    model: classifier
    predictors: independent variables
    target: dependent variable
    """

    # predicting using the independent variables
    pred = model.predict(predictors)

    acc = accuracy_score(target, pred)  # to compute Accuracy
    recall = recall_score(target, pred)  # to compute Recall
    precision = precision_score(target, pred)  # to compute Precision
    f1 = f1_score(target, pred)  # to compute F1-score

    # creating a dataframe of metrics
    df_perf = pd.DataFrame(
        {"Accuracy": acc, "Recall": recall, "Precision": precision, "F1": f1,},
        index=[0],
    )

    return df_perf

In [ ]:
def confusion_matrix_sklearn(model, predictors, target):
    """
    To plot the confusion_matrix with percentages

    model: classifier
    predictors: independent variables
    target: dependent variable
    """
    y_pred = model.predict(predictors)
    cm = confusion_matrix(target, y_pred)
    labels = np.asarray(
        [
            ["{0:0.0f}".format(item) + "\n{0:.2%}".format(item / cm.flatten().sum())]
            for item in cm.flatten()
        ]
    ).reshape(2, 2)

    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=labels, fmt="")
    plt.ylabel("True label")
    plt.xlabel("Predicted label")

#### Defining scorer to be used for cross-validation and hyperparameter tuning

In [ ]:
# Type of scoring used to compare parameter combinations is f1_score
scorer = metrics.make_scorer(metrics.f1_score)

**We are now done with pre-processing and evaluation criterion, so let's start building the model.**

### Model building with original data

In [ ]:
models = []  # Empty list to store all the models

# Appending models into the list
models.append(("Bagging", BaggingClassifier(estimator=DecisionTreeClassifier(random_state=1, class_weight='balanced'), random_state=1)))
models.append(("Random forest", RandomForestClassifier(random_state=1, class_weight='balanced')))
models.append(("GBM", GradientBoostingClassifier(random_state=1)))
models.append(("Adaboost", AdaBoostClassifier(random_state=1)))
models.append(("dtree", DecisionTreeClassifier(random_state=1, class_weight='balanced')))

results1 = []  # Empty list to store all model's CV scores
names = []  # Empty list to store name of the models


# loop through all models to get the mean cross validated score
print("\n" "Cross-Validation performance on training dataset:" "\n")

for name, model in models:
    kfold = StratifiedKFold(
        n_splits=10, shuffle=True, random_state=1
    )
    cv_result = cross_val_score(
        estimator=model, X=X_train, y=y_train, scoring = scorer,cv=kfold
    )
    results1.append(cv_result)
    names.append(name)
    print("{}: {}".format(name, cv_result.mean()))

print("\n" "Validation Performance:" "\n")

for name, model in models:
    model.fit(X_train, y_train)
    scores = f1_score(y_val, model.predict(X_val))
    print("{}: {}".format(name, scores))

In [ ]:
# Plotting boxplots for CV scores of all models defined above
fig = plt.figure(figsize=(10, 7))

fig.suptitle("Algorithm Comparison")
ax = fig.add_subplot(111)

plt.boxplot(results1)
ax.set_xticklabels(names)

plt.show()

* GBM has the best performance followed by AdaBoost model as per the validation performance and then Random forest algorithms !

### Model Building with oversampled data

In [ ]:
print("Before OverSampling, counts of label '1': {}".format(sum(y_train == 1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train == 0)))

# Synthetic Minority Over Sampling Technique
sm = SMOTE(sampling_strategy=1, k_neighbors=5, random_state=1)
X_train_over, y_train_over = sm.fit_resample(X_train, y_train)


print("After OverSampling, counts of label '1': {}".format(sum(y_train_over == 1)))
print("After OverSampling, counts of label '0': {} \n".format(sum(y_train_over == 0)))


print("After OverSampling, the shape of train_X: {}".format(X_train_over.shape))
print("After OverSampling, the shape of train_y: {} \n".format(y_train_over.shape))

* Now we see, as part of Oversampling, the number of cases Certified and Denied are exactly the same as at 11913 records when done with sampling_strategy = 1 and k_neighbours = 5 as input.

In [ ]:
models = []  # Empty list to store all the models

# Appending models into the list for Oversampled data
models.append(("Bagging", BaggingClassifier(estimator=DecisionTreeClassifier(random_state=1, class_weight='balanced'), random_state=1)))
models.append(("Random forest", RandomForestClassifier(random_state=1, class_weight='balanced')))
models.append(("GBM", GradientBoostingClassifier(random_state=1)))
models.append(("Adaboost", AdaBoostClassifier(random_state=1)))
models.append(("dtree", DecisionTreeClassifier(random_state=1, class_weight='balanced')))

results1 = []  # Empty list to store all model's CV scores
names = []  # Empty list to store name of the models


# loop through all models to get the mean cross validated score
print("\n" "Cross-Validation performance on training dataset for Oversampled data:" "\n")

for name, model in models:
    kfold = StratifiedKFold(
        n_splits=10, shuffle=True, random_state=1
    )
    cv_result = cross_val_score(
        estimator=model, X=X_train_over, y=y_train_over,scoring = scorer, cv=kfold
    )
    results1.append(cv_result)
    names.append(name)
    print("{}: {}".format(name, cv_result.mean()))

print("\n" "Validation Performance for Oversampled data:" "\n")

for name, model in models:
    model.fit(X_train_over, y_train_over)
    scores = f1_score(y_val, model.predict(X_val))
    print("{}: {}".format(name, scores))

In [ ]:
# Plotting boxplots for CV scores of all models defined above
fig = plt.figure(figsize=(10, 7))

fig.suptitle("Algorithm Comparison for Oversampled data")
ax = fig.add_subplot(111)

plt.boxplot(results1)
ax.set_xticklabels(names)

plt.show()

* GBM has the best performance followed by AdaBoost model as per the validation performance and then Random forest algorithms !

### Model Building with undersampled data

In [ ]:
rus = RandomUnderSampler(random_state=1, sampling_strategy=1)
X_train_un, y_train_un = rus.fit_resample(X_train, y_train)


print("Before UnderSampling, counts of label '1': {}".format(sum(y_train == 1)))
print("Before UnderSampling, counts of label '0': {} \n".format(sum(y_train == 0)))


print("After UnderSampling, counts of label '1': {}".format(sum(y_train_un == 1)))
print("After UnderSampling, counts of label '0': {} \n".format(sum(y_train_un == 0)))


print("After UnderSampling, the shape of train_X: {}".format(X_train_un.shape))
print("After UnderSampling, the shape of train_y: {} \n".format(y_train_un.shape))

* Now as we can see after undersampling, the number of denied cases become same as that of Certified cases, meaning Certified values are lost to achieve this match. In ideal Business environment, we should not be performing Undersampling !

In [ ]:
models = []  # Empty list to store all the models

# Appending models into the list for Oversampled data
models.append(("Bagging", BaggingClassifier(estimator=DecisionTreeClassifier(random_state=1, class_weight='balanced'), random_state=1)))
models.append(("Random forest", RandomForestClassifier(random_state=1, class_weight='balanced')))
models.append(("GBM", GradientBoostingClassifier(random_state=1)))
models.append(("Adaboost", AdaBoostClassifier(random_state=1)))
models.append(("dtree", DecisionTreeClassifier(random_state=1, class_weight='balanced')))

results1 = []  # Empty list to store all model's CV scores
names = []  # Empty list to store name of the models


# loop through all models to get the mean cross validated score
print("\n" "Cross-Validation performance on training dataset of Undersampled data:" "\n")

for name, model in models:
    kfold = StratifiedKFold(
        n_splits=10, shuffle=True, random_state=1
    )
    cv_result = cross_val_score(
        estimator=model, X=X_train_un, y=y_train_un,scoring = scorer, cv=kfold
    )
    results1.append(cv_result)
    names.append(name)
    print("{}: {}".format(name, cv_result.mean()))

print("\n" "Validation Performance of Undersampled data:" "\n")

for name, model in models:
    model.fit(X_train_un, y_train_un)
    scores = f1_score(y_val, model.predict(X_val))
    print("{}: {}".format(name, scores))

In [ ]:
# Plotting boxplots for CV scores of all models defined above
fig = plt.figure(figsize=(10, 7))

fig.suptitle("Algorithm Comparison of Undersampled data")
ax = fig.add_subplot(111)

plt.boxplot(results1)
ax.set_xticklabels(names)

plt.show()

* GBM has the best performance followed by AdaBoost model as per the validation performance and then Random forest algorithms !




- After building 15 models, it was observed that both the GBM and Adaboost models, trained on an undersampled dataset is underfitting in general, while the GBM model and Adaboost trained on an oversampled dataset, exhibited strong performance on both the training and validation datasets.
- Sometimes models might underfit/overfit after undersampling/oversampling respectively, so it's better to tune the models to get a generalized performance
- We will tune these 3 models using the same data (undersampled or oversampled) as we trained them on before !

## Hyperparameter Tuning

### Tuning AdaBoost using oversampled data

In [ ]:
%%time

# defining model
Model = AdaBoostClassifier(random_state=1)

# Parameter grid to pass in RandomSearchCV
param_grid = {
    "n_estimators": np.arange(10, 40, 10),
    "learning_rate": [0.1, 0.01, 0.2, 0.05, 1],
    "estimator": [DecisionTreeClassifier(max_depth=1, random_state=1), DecisionTreeClassifier(max_depth=2, random_state=1), DecisionTreeClassifier(max_depth=3, random_state=1),
    ]
}


#Calling RandomizedSearchCV
randomized_cv = RandomizedSearchCV(estimator=Model, param_distributions=param_grid, n_iter=50, scoring=scorer, cv=5, random_state=1)  ## Complete the code to set the cv parameter

#Fitting parameters in RandomizedSearchCV
randomized_cv.fit(X_train_over, y_train_over) ## code to fit the model on over sampled data

print("Best parameters are {} with CV score={}:" .format(randomized_cv.best_params_,randomized_cv.best_score_))

In [ ]:
## Complete the code to set the best parameters.
tuned_ada = AdaBoostClassifier(
    n_estimators= 30, learning_rate= 1, estimator= DecisionTreeClassifier(max_depth=2, random_state=1)
)

tuned_ada.fit(X_train_over, y_train_over)

In [ ]:
ada_train_perf = model_performance_classification_sklearn(tuned_ada, X_train_over, y_train_over)
ada_train_perf

In [ ]:
ada_val_perf = model_performance_classification_sklearn(tuned_ada,X_val, y_val)
ada_val_perf

### Tuning Random forest using undersampled data

In [ ]:
%%time

# defining model
Model = RandomForestClassifier(random_state=1)

# Parameter grid to pass in RandomSearchCV
param_grid = {
    "n_estimators": [150,200,250],
    "min_samples_leaf": np.arange(5, 10),
    "max_features": np.arange(0.2, 0.7, 0.1),
    "max_samples": np.arange(0.3, 0.7, 0.1),
     }

#Calling RandomizedSearchCV
randomized_cv = RandomizedSearchCV(estimator=Model, param_distributions=param_grid, n_iter=50, scoring=scorer, cv=5, random_state=1) ## Complete the code to set the cv parameter

#Fitting parameters in RandomizedSearchCV
randomized_cv.fit(X_train_un, y_train_un)

print("Best parameters are {} with CV score={}:" .format(randomized_cv.best_params_,randomized_cv.best_score_))

In [ ]:
tuned_rf2 = RandomForestClassifier(
    max_features=0.2,
    random_state=1,
    max_samples=0.4,
    n_estimators=250,
    min_samples_leaf=9,
)

tuned_rf2.fit(X_train_un, y_train_un)

In [ ]:
 # Checking model's performance on training set
rf2_train_perf = model_performance_classification_sklearn(
    tuned_rf2, X_train_un, y_train_un
)
rf2_train_perf

In [ ]:
# code to print the model performance on the validation data.
rf2_val_perf = model_performance_classification_sklearn(tuned_rf2,X_val, y_val)
rf2_val_perf

### Tuning with Gradient boosting with oversampled data

In [ ]:
%%time

# defining model
Model = GradientBoostingClassifier(random_state=1)

## Complete the code to define the hyper parameters.
param_grid={
    "n_estimators": np.arange(75,150,25),
    "learning_rate": [0.1, 0.01, 0.2, 0.05, 1],
    "subsample":[0.5,0.7,1],
    "max_features":[0.5,0.7,1],
}

## Complete the code to set the cv parameter.
randomized_cv = RandomizedSearchCV(estimator=Model, param_distributions=param_grid, scoring=scorer, n_iter=50, cv=5, random_state=1)

#Fitting parameters in RandomizedSearchCV
randomized_cv.fit(X_train_over, y_train_over)

print("Best parameters are {} with CV score={}:" .format(randomized_cv.best_params_,randomized_cv.best_score_))

In [ ]:
## Code to define the best model.
tuned_gbm = GradientBoostingClassifier(
    max_features=0.5,
    random_state=1,
    learning_rate=0.2,
    n_estimators=100,
    subsample=1
)

tuned_gbm.fit(X_train_over, y_train_over)

In [ ]:
 # Checking model's performance on training set
 gbm_train_perf = model_performance_classification_sklearn(
    tuned_gbm, X_train_over, y_train_over
)
gbm_train_perf

In [ ]:
## Code to print the model performance on the validation data.
gbm_val_perf = model_performance_classification_sklearn(tuned_gbm,X_val, y_val)
gbm_val_perf

## Model Performances

In [ ]:
# training performance comparison

models_train_comp_df = pd.concat(
    [
        gbm_train_perf.T,
        ada_train_perf.T,
        rf2_train_perf.T,
    ],
    axis=1,
)
models_train_comp_df.columns = [
    "Gradient Boosting tuned with oversampled data",
    "AdaBoost tuned with oversampled data",
    "Random forest tuned with undersampled data",
]
print("Training performance comparison:")
models_train_comp_df

In [ ]:
# validation performance comparison

models_val_comp_df = pd.concat(
    [
        gbm_val_perf.T,
        ada_val_perf.T,
        rf2_val_perf.T,
    ],
    axis=1,
)
models_val_comp_df.columns = [
    "Gradient Boosting tuned with oversampled data",
    "AdaBoost tuned with oversampled data",
    "Random forest tuned with undersampled data",
]
print("Validation performance comparison:")
models_val_comp_df

- Gradient Boosting tuned with oversampled data has generalised performance, so let's consider it as the best model in terms of better F1 score metric !

In [ ]:
## Code to print the model performance on the test data by the best model.
test = model_performance_classification_sklearn(tuned_gbm, X_test, y_test)
test

- The Gradient model trained on oversampled data has given ~83% F1 score on the test set
- This performance is in line and is even better than with what we achieved with this model on the train and validation sets, hence this model neither overfit or underfit and it rather generalizes model !!

### Feature Importance

In [ ]:
# Feature importance identification based on the best model tuned which is the Gradient model trained
feature_names = X_train.columns
importances = tuned_gbm.feature_importances_
indices = np.argsort(importances)

plt.figure(figsize=(12, 12))
plt.title("Feature Importances")
plt.barh(range(len(indices)), importances[indices], color="violet", align="center")
plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
plt.xlabel("Relative Importance")
plt.show()

- The top 5 important features to be considered while certifying the Visa for an applicant are listed in the below order:-

1. Education of the Employee
2. Region of Employment in the USA
3. Continent of the Applicant
4. Prevailing wages and therefore any amount listed in unit of wage as (yearly) takes precedence (and)
5. The Applicant has previous job experience


## Actionable Insights and Recommendations

---> Actionable Insights from selected ML Model:

- The dataset consisted on 25480 rows and 12 columns with data on Visa application and it's status for USA country - we built multiple models to help OFLC predict the chances on Visa Application being certified or denied efficiently.
- Tuned Gradient Boosting model was selected as the final model due to the performance scores and feasiblity of using the model, on F1 score metric.
- We consider that False negatives and false positives both have higher cost on the execution of the model. Therefore, we chose F1 score as our metric for a balanced solution to visa approvals.
- We tried to optimize the metric on all our models, and concluded that tuned gradient boosting classifier was the best performing model with an F1 score of ~83% on the test sets and ~ 81% on the test / validation sets of data, which do not overfit as well.

---> EDA insights are clearly listed out in the line number [16] in this Notebook.

Feature importance analysis done above is such a critical outcome of this exercise, on top of the tuned Gradient Boosting classifier as listed above and the OFLC could screen application at different levels:-

1) Education Level (Higher the education level, with Doctorate -> Masters -> Bachelors and then High School) better the changes for Visa Certification in the order listed.
2) Job experience - should have prior job experience for higher chance of Visa to be certified.
3) Previaling Wage - when gauged annually, with a median of above > 72000 USD, the chances of approval are higher, while people with hourly unit of wage have higher chances of Visa being denied.
4) Continent from where the Applicant hails from : Applicants from Europe, Africa and Asia in this order have had higher chances of Visa being certified.
5) Region where they would intend to work: Mid-West region have higher chances of Visa approval.

The converse cases of the above observations are generally deemed to have the Visa application denied, which OFLC can use as a first hand mode of elimination, in conjunction with our Machine Learning Model.

---> Business Recommendations:-

- More information on the Employee skill level / specialization is missing
- Employees experience would have been key field for gathering rather than the years of establishment of the Organisation, in which the employee works
- It would be interesting to know the demand in the job market for workforce needed along with the skillsets, which could be analyzed along with applicant's skillset fitment.
- Overall, the model generalised well in the Test data and we could recommend the Business to use the model along with the above recommendations for further fine tuning the performance of the model.


<font size=6 color='blue'>Power Ahead</font>
___